In [1]:
import torch
from torch import nn

import torchvision
from torchvision import transforms

from torchinfo import summary

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
train_dir = "../data/chest_xray/train/"
test_dir = "../data/chest_xray/test/"

In [4]:
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.ColorJitter(),
    transforms.RandomRotation(degrees=15),
])

In [5]:
from pytorch.torchvision import dataloader

In [6]:
train_dataloader, test_dataloader, class_names = dataloader.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=data_transform,
                                                                               test_transform=data_transform,
                                                                               batch_size=32,
                                                                               )
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1cfe683c9a0>,
 ['NORMAL', 'PNEUMONIA'])

In [7]:
class pneumoniav0(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.conv2d_1 = nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
        self.conv2d_2 = nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units*2, kernel_size=3, stride=1, padding=1)
        self.conv2d_3 = nn.Conv2d(in_channels=hidden_units*2, out_channels=hidden_units*3, kernel_size=3, stride=1, padding=1)
        self.conv2d_4 = nn.Conv2d(in_channels=hidden_units*3, out_channels=hidden_units*4, kernel_size=3, stride=1, padding=1)
        self.maxpool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear_1 = nn.Linear(in_features=hidden_units*4*(16*16), out_features=output_shape)

    def forward(self, x):

        z = self.conv2d_1(x)
        z = self.relu(z)
        z = self.conv2d_2(z)
        z = self.relu(z)
        z = self.maxpool_1(z)
        z = self.conv2d_3(z)
        z = self.relu(z)
        z = self.conv2d_4(z)
        z = self.relu(z)
        z = self.maxpool_1(z)
        z = self.flatten(z)
        z = self.linear_1(z)

        return z

In [8]:
model_0 = pneumoniav0(input_shape=3,
                      hidden_units=20,
                      output_shape=1).to(device)
model_0

pneumoniav0(
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv2d_1): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2d_2): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2d_3): Conv2d(40, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2d_4): Conv2d(60, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_1): Linear(in_features=20480, out_features=1, bias=True)
)

In [9]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)

In [10]:
from pytorch.torchvision import set_seed, writer
from pytorch.torchvision.classification import binary

In [11]:
set_seed.set_seeds()

epochs = 5


results = binary.train(model=model_0,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 optimizer=optimizer,
                 loss_fn=loss_fn,
                 epochs=epochs,
                 device=device,
                 writer=writer.create_writer(experiment_name="rough_doc",
                                             model_name="model_0",
                                             extra=f"epochs_{epochs}"))

[INFO] created SummaryWriter, saving to: runs\2024_02_14_14_24_14\rough_doc\model_0\epochs_5...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5631 | train_acc: 74.0473 | test_loss: 0.6523 | test_acc: 63.4375
Epoch: 2 | train_loss: 0.5384 | train_acc: 75.1524 | test_loss: 0.7387 | test_acc: 63.4375
Epoch: 3 | train_loss: 0.5243 | train_acc: 77.2675 | test_loss: 1.0003 | test_acc: 63.4375
Epoch: 4 | train_loss: 0.3811 | train_acc: 83.6700 | test_loss: 0.4704 | test_acc: 76.8750
Epoch: 5 | train_loss: 0.2387 | train_acc: 90.3582 | test_loss: 0.4585 | test_acc: 80.7812


In [13]:
from pytorch.torchvision import save_model

In [14]:
save_model.save_model(model=model_0,target_dir="models",model_name="pneumonialungs_2_model_0_5_epochs.pth")

[INFO] Saving model to: models\pneumonialungs_2_model_0_5_epochs.pth


In [15]:
weights = torchvision.models.EfficientNet_B7_Weights.DEFAULT
weights

EfficientNet_B7_Weights.IMAGENET1K_V1

In [16]:
auto_transform = weights.transforms()
auto_transform

ImageClassification(
    crop_size=[600]
    resize_size=[600]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [ ]:
train_dataloader, test_dataloader, class_names = dataloader.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=auto_transform,
                                                                               test_transform=auto_transform,
                                                                               batch_size=32,
                                                                               )
train_dataloader, test_dataloader, class_names

In [17]:
model_1 = torchvision.models.efficientnet_b7(weights=weights).to(device)
model_1

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to C:\Users\udayp/.cache\torch\hub\checkpoints\efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:39<00:00, 6.71MB/s]   


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [18]:
summary(model=model_1,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 2560, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 64, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 64, 112, 112]   1,728                True
│    │    └─BatchNorm2d (1)                                  [32, 64, 112, 112]   [32, 64, 112, 112]   128                  True
│    │    └─SiLU (2)                                         [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 64, 112, 112]   [32, 32, 112

In [19]:
for param in model_1.features.parameters():
    param.requires_grad = False

(classifier): Sequential(
    (0): Dropout(p=0.5, inplace=True)
    (1): Linear(in_features=2560, out_features=1000, bias=True)
  )

In [20]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)

model_1.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.5, inplace=True),
    torch.nn.Linear(in_features=2560,
                    out_features=1,
                    bias=True)
).to(device)

In [21]:
summary(model=model_1,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 2560, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 64, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 64, 112, 112]   (1,728)              False
│    │    └─BatchNorm2d (1)                                  [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
│    │    └─SiLU (2)                                         [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 64, 112, 112]   [32, 

In [22]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.1)

In [23]:
set_seed.set_seeds()

epochs = 5


results = binary.train(model=model_1,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 optimizer=optimizer,
                 loss_fn=loss_fn,
                 epochs=epochs,
                 device=device,
                 writer=writer.create_writer(experiment_name="rough_doc_transfer",
                                             model_name="model_1",
                                             extra=f"epochs_{epochs}"))

[INFO] created SummaryWriter, saving to: runs\2024_02_14_14_45_15\rough_doc_transfer\model_1\epochs_5...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.4516 | train_acc: 78.9062 | test_loss: 0.7189 | test_acc: 63.5938
Epoch: 2 | train_loss: 0.3869 | train_acc: 82.4695 | test_loss: 0.7945 | test_acc: 66.4062
Epoch: 3 | train_loss: 0.4050 | train_acc: 82.6029 | test_loss: 0.7326 | test_acc: 66.8750
Epoch: 4 | train_loss: 0.4050 | train_acc: 82.7363 | test_loss: 0.7266 | test_acc: 66.5625
Epoch: 5 | train_loss: 0.4197 | train_acc: 81.5549 | test_loss: 0.6114 | test_acc: 71.5625


In [24]:
set_seed.set_seeds()

epochs = 5


results = binary.train(model=model_1,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 optimizer=optimizer,
                 loss_fn=loss_fn,
                 epochs=epochs,
                 device=device,
                 writer=writer.create_writer(experiment_name="rough_doc_transfer",
                                             model_name="model_1",
                                             extra=f"epochs_{epochs}"))

[INFO] created SummaryWriter, saving to: runs\2024_02_14_15_12_42\rough_doc_transfer\model_1\epochs_5...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.3781 | train_acc: 83.7652 | test_loss: 0.5886 | test_acc: 72.3438
Epoch: 2 | train_loss: 0.3617 | train_acc: 83.8796 | test_loss: 0.6798 | test_acc: 72.5000
Epoch: 3 | train_loss: 0.3833 | train_acc: 83.7652 | test_loss: 0.5889 | test_acc: 73.2812
Epoch: 4 | train_loss: 0.3875 | train_acc: 83.8796 | test_loss: 0.6241 | test_acc: 72.8125
Epoch: 5 | train_loss: 0.3995 | train_acc: 82.5267 | test_loss: 0.6672 | test_acc: 70.3125


In [25]:
save_model.save_model(model=model_1,target_dir="models",model_name="pneumonialungs_2_model_1_10_epochs.pth")

[INFO] Saving model to: models\pneumonialungs_2_model_1_10_epochs.pth


In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

In [ ]:
auto_transform = weights.transforms()
auto_transform

In [ ]:
train_dataloader, test_dataloader, class_names = dataloader.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=auto_transform,
                                                                               test_transform=auto_transform,
                                                                               batch_size=32,
                                                                               )
train_dataloader, test_dataloader, class_names

In [ ]:
model_2 = torchvision.models.efficientnet_b0(weights=weights).to(device)
model_2

In [ ]:
summary(model=model_2,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

In [ ]:
for param in model_2.features.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)

model_2.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=1,
                    bias=True)
).to(device)

In [ ]:
summary(model=model_2,
        input_size=[32,3,224,224],
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(),
                            lr=0.1)

In [ ]:
set_seed.set_seeds()

epochs = 10

results = binary.train(model=model_2,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 optimizer=optimizer,
                 loss_fn=loss_fn,
                 epochs=epochs,
                 device=device,
                 writer=writer.create_writer(experiment_name="rough_doc_transfer",
                                             model_name="model_1",
                                             extra=f"epochs_{epochs}"))